In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

# Implement Action-history-dependent Learning Algorithm

In [2]:
# Problem data.
# Input: n, d1,...,dm

# m is the number of variables
m = 4

# n is the number of time steps
# (constraints that are revealed one by one)
n_values = [25, 50, 100, 250, 500, 1000, 2000]

d = np.full(m, 0.25)

In [3]:
# Make results deterministic for debugging
np.random.seed(23)

In [4]:
# Runs Algorithm 3 in paper [5]
def run_alg_3(n, m, d):
    start = time.time()
    
    
    coin_flips = np.random.randint(2, size=m*n)
    A = coin_flips.reshape((m,n))
    pi = np.random.randint(2, size=n)

    # Initialize the constraint b_i0 = n d_i for i=1,...,m
    b = n*d

    historical_b = np.zeros((n,m))
    historical_b[0] = b

    # Initialize the dual price p1 = 0

    p = np.zeros((m,))

    historical_p = np.zeros((n,m))
    historical_p[0] = p

    # Initialize x vector
    x = np.ndarray(0)

    ############################################################

    # For t=1,...,n do
    for t in range(1,n):
        if t % 500 == 0:
            now = time.time()
            print("%d iterations completed in %.2f s" % (t, now-start))
        
        #print("\nIteration %d" % (t))
        # Observe (pi_t, a_t)
        a_t = A[:,t-1]
        pi_t = pi[t-1]
        #print("a_t: %s" % (a_t))
        #print("pi_t: %s" % (pi_t))

        # Set:
        # x_t = 1 if r_t > a_t' p_t
        # x_t = 0 if r_t <=  a_t' p_t
        # Save values of x into array
        xt = 0
        if pi_t > np.transpose(a_t)@historical_p[t-1]:
            xt = 1
        x = np.append(x, xt)
        #print("x: %s" % (x))

        # Update the constraint vector
        b = historical_b[t-1] - np.array(a_t*xt).flatten()
        historical_b[t] = b
        #print("b: %s" % (b))

        # Specify a primal optimization problem
        # max sum from j=1 to t of r_j x_j
        # such that sum from j=1 to t of a_ij x_j <= tb_{it} / (n-t) for i=1,...,m
        # 0<=xj<=1 for j=1,...,t

        # Define variables
        a_up_to_t = A[:,:t]
        pi_up_to_t = pi[:t]
        x_opt = cp.Variable(t)

        # Create constraints.
        constraints = [a_up_to_t @ x_opt <= t*b / (n-t),
                       x_opt >= 0,
                       x_opt <= 1]

        # Form objective.
        obj = cp.Maximize(pi_up_to_t @ x_opt)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()

        # The optimal dual variable (Lagrange multiplier) for
        # a constraint is stored in constraint.dual_value.
        #print("optimal (a_up_to_t @ x_opt <= t*b / (n-t)) dual variable", constraints[0].dual_value)

        p = constraints[0].dual_value

        historical_p[t] = p

    return historical_b

## Use the setting of Random Input I to assess stopping time and remaining inventory B_n

In [6]:
num_trials = 10

In [22]:
def get_minimum_resources_arr(hist_b):
    return hist_b.min(axis=1)
    
def compute_stopping_time(min_resources_arr, s):
    tau = np.argmax(min_resources_arr<s)
    if tau == 0:
        return len(min_resources_arr)
    return tau

def compute_avg_of_dict_vals(this_dict):
    return np.mean(list(this_dict.values()))

In [32]:
results_df = pd.DataFrame(columns=['n', 'tau_b/2', 'tau_b/4', 'tau_0', 'b_n'])


# This dictionary maps n to an array of dictionaries
# for each trial, we save the results:
# [tn_to_hist_b, tn_to_min_resources_arr,
# tn_to_stopping_time_b2, tn_to_stopping_time_b4,
# tn_to_stopping_time_0, tn_to_bn]
n_to_tn_results = dict()

for n in n_values:
    
    # tn is the number of trials
    tn_to_hist_b = dict()
    tn_to_min_resources_arr = dict()
    tn_to_stopping_time_b2 = dict()
    tn_to_stopping_time_b4 = dict()
    tn_to_stopping_time_0 = dict()
    tn_to_bn = dict()
    
    for tn in range(num_trials):
        print("n=%d\t Trial %d" % (n, tn))
        hist_b = run_alg_3(n, m, d)
        min_resources_arr = get_minimum_resources_arr(hist_b)
        stopping_time_b2 = compute_stopping_time(min_resources_arr, n*0.25/2)
        stopping_time_b4 = compute_stopping_time(min_resources_arr, n*0.25/4)
        stopping_time_0 = compute_stopping_time(min_resources_arr, 0)
        bn = min(hist_b[-1])

        tn_to_hist_b[tn] = hist_b
        tn_to_min_resources_arr[tn] = min_resources_arr
        tn_to_stopping_time_b2[tn] = stopping_time_b2
        tn_to_stopping_time_b4[tn] = stopping_time_b4
        tn_to_stopping_time_0[tn] = stopping_time_0
        tn_to_bn[tn] = bn

        #print("T_{b/2} %d\t T_{b/4} %d\t T_0 %d\t B_n %.2f" %
        #      (stopping_time_b2, stopping_time_b4,
        #      stopping_time_0, bn))

    print("Average T_{b/2} %d\t T_{b/4} %d\t T_0 %d\t B_n %.2f" %
              (compute_avg_of_dict_vals(tn_to_stopping_time_b2),
               compute_avg_of_dict_vals(tn_to_stopping_time_b4),
               compute_avg_of_dict_vals(tn_to_stopping_time_0),
               compute_avg_of_dict_vals(tn_to_bn)
              )
         )

    results_df = results_df.append({
        'n': n,
        'tau_b/2': compute_avg_of_dict_vals(tn_to_stopping_time_b2),
        'tau_b/4': compute_avg_of_dict_vals(tn_to_stopping_time_b4),
        'tau_0': compute_avg_of_dict_vals(tn_to_stopping_time_0),
        'b_n': compute_avg_of_dict_vals(tn_to_bn)
    }, ignore_index = True)
    
    n_to_tn_results[n] = [tn_to_hist_b,
                                 tn_to_min_resources_arr,
                                 tn_to_stopping_time_b2,
                                 tn_to_stopping_time_b4,
                                 tn_to_stopping_time_0,
                                 tn_to_bn
                                ]

n=25	 Trial 0
n=25	 Trial 1
n=25	 Trial 2
n=25	 Trial 3
n=25	 Trial 4
n=25	 Trial 5
n=25	 Trial 6
n=25	 Trial 7
n=25	 Trial 8
n=25	 Trial 9
Average T_{b/2} 11	 T_{b/4} 17	 T_0 23	 B_n 0.17
n=50	 Trial 0
n=50	 Trial 1
n=50	 Trial 2
n=50	 Trial 3
n=50	 Trial 4
n=50	 Trial 5
n=50	 Trial 6
n=50	 Trial 7
n=50	 Trial 8
n=50	 Trial 9
Average T_{b/2} 26	 T_{b/4} 37	 T_0 48	 B_n 0.12
n=100	 Trial 0
n=100	 Trial 1
n=100	 Trial 2
n=100	 Trial 3
n=100	 Trial 4
n=100	 Trial 5
n=100	 Trial 6
n=100	 Trial 7
n=100	 Trial 8
n=100	 Trial 9
Average T_{b/2} 48	 T_{b/4} 77	 T_0 100	 B_n 2.07
n=250	 Trial 0
n=250	 Trial 1
n=250	 Trial 2
n=250	 Trial 3
n=250	 Trial 4
n=250	 Trial 5
n=250	 Trial 6
n=250	 Trial 7
n=250	 Trial 8
n=250	 Trial 9
Average T_{b/2} 125	 T_{b/4} 190	 T_0 249	 B_n 1.30
n=500	 Trial 0
n=500	 Trial 1
n=500	 Trial 2
n=500	 Trial 3
n=500	 Trial 4
n=500	 Trial 5
n=500	 Trial 6
n=500	 Trial 7
n=500	 Trial 8
n=500	 Trial 9
Average T_{b/2} 259	 T_{b/4} 383	 T_0 499	 B_n 1.03
n=1000	 Trial 0
50

In [33]:
results_df

,n,tau_b/2,tau_b/4,tau_0,b_n
0,25.0,11.9,17.7,23.6,0.166590
1,50.0,26.0,37.8,48.8,0.115954
2,100.0,48.8,77.0,100.0,2.068300
3,250.0,125.9,190.1,249.4,1.295236
4,500.0,259.0,383.8,499.7,1.032516
5,1000.0,505.8,758.6,999.1,1.925916
6,2000.0,1007.3,1517.0,2000.0,6.486894
